<a href="https://colab.research.google.com/github/rprimi/b5_nlp/blob/main/m2_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

install.packages("devtools")
devtools::install_github("rstudio/reticulate")
library(reticulate)
use_python("/usr/local/lib/python3.10")


install.packages("tensorflow")
library(tensorflow)
install.packages("keras")
library(keras)


library(reticulate)
use_python("/usr/bin/python3")


install_tensorflow(
  version = "default",
  method = c("auto"),
  cuda = TRUE,
 envname = "NULL" )




In [9]:
pacakges <- c("tidyverse", "keras", "psych", "readr", "zip")
install.packages(pacakges)

library(tidyverse)
library(keras)
library(psych)
library(readr)
library(zip)


tensorflow::tf_version()
tensorflow::tf_config()
tensorflow::tf_gpu_configured(verbose = TRUE)

TensorFlow built with CUDA:  TRUE 
GPU device name:  /device:GPU:0

[1] TRUE

In [3]:
load("base_bfi_nlp.RData")

In [10]:
samples <- db_textos_splited$texto_dividido

tokenizer <- text_tokenizer(
    lower = TRUE )  %>%
    fit_text_tokenizer(samples)

word_index <- tokenizer$word_index

sequences <- texts_to_sequences(tokenizer, samples)

cat("Found", length(word_index), "unique tokens.\n")



Found 66128 unique tokens.


In [ ]:
load("/content/vocabulary.RData")
names(vocabulary)

# Function adapted from Chalot book
  prepare_embedding_matrix <- function(num_words, EMBEDDING_DIM, word_index) {
      MAX_NUM_WORDS = num_words
      embedding_matrix <- matrix(0L, nrow = num_words+1, ncol = EMBEDDING_DIM)
      for (word in names(word_index)) {
        index <- word_index[[word]]
        if (index >= MAX_NUM_WORDS)
          next
        embedding_vector <- as.numeric(vocabulary[vocabulary$word == word, 3:602])
        if (!is.null(embedding_vector)) {
          # words not found in embedding index will be all-zeros.
          embedding_matrix[index+1,] <- embedding_vector
        }
      }
      embedding_matrix
    }

# Creates embedding matrix
     embedding_matrix <- prepare_embedding_matrix(
         num_words = 66128,
         EMBEDDING_DIM = 600,
         word_index = word_index)

# Fills empty rows with random number
    rnd <- runif(n=sum(is.na(embedding_matrix)), min = 0, max = .04)
    embedding_matrix[is.na(embedding_matrix)] <- rnd

# Test it
   table(is.na(embedding_matrix))

# See shape of embedding matrix and first row (needs to be zero, don't know why)
# https://github.com/rstudio/keras/issues/302)
   dim(embedding_matrix)
   embedding_matrix[1, ]


In [ ]:
# Shuffle data
    hist(map_dbl(sequences, length))
    set.seed(8)

    indices <- sample(1:length(sequences))

    prop_train <- .82
    maxlen <-250

    train_indices <- 1: round(prop_train*length(sequences), 0)
    test_indices <-  (round(prop_train*length(sequences), 0)+1) : length(sequences)


    x_train <- pad_sequences(sequences[indices[train_indices]], maxlen = maxlen)
    x_test <- pad_sequences(sequences[indices[test_indices]], maxlen = maxlen)

    db_ys <- db_textos_splited[ , "id"] %>%
      as.data.frame() %>%
      set_names("id") %>%
      mutate(id = as.numeric(id)) %>%
      left_join( {db_bfi %>% select(id, O_rec:N_vlti_rec)}, by = "id" )


    y_train <- as.matrix(db_ys[indices[train_indices] , 2:6])
    y_test <-  as.matrix(db_ys[indices[test_indices] , 2:6])

    dim(x_train)
    dim(x_test)

    dim(y_train)
    dim(y_test)


In [16]:
  max_words = 66128
    embedding_dim = 600

# bidirectional lstm model

   model <- keras_model_sequential() %>%
     layer_embedding(
         input_dim = max_words + 1,
         output_dim = embedding_dim ,
         weights = list(embedding_matrix),
         input_length = maxlen,
         trainable = FALSE ) %>%
    bidirectional(layer_lstm(units = 64,recurrent_dropout = 0.5, dropout =.5)) %>%
    layer_dense(units = 5,  kernel_regularizer = regularizer_l2(0.001))

    summary(model)


Model: "sequential"
________________________________________________________________________________
 Layer (type)                  Output Shape               Param #    Trainable  
 embedding (Embedding)         (None, 250, 600)           39677400   N          
 bidirectional (Bidirectional  (None, 128)                340480     Y          
 )                                                                              
 dense (Dense)                 (None, 5)                  645        Y          
Total params: 40018525 (152.66 MB)
Trainable params: 341125 (1.30 MB)
Non-trainable params: 39677400 (151.36 MB)
________________________________________________________________________________


In [33]:
class_callback_loss_history <- new_callback_class(
    "loss_history",
    initialize = function() {
      self$batch_losses <- numeric()
    },
    on_batch_end = function(batch, logs) {
      # Append current batch's loss to the list
      self$batch_losses <- c(self$batch_losses, logs$get("loss"))

      # Plot the losses
      df <- data.frame(Batch = 1:length(self$batch_losses), Loss = self$batch_losses)
      print(
        ggplot(df, aes(x = Batch, y = Loss)) +
          geom_line(color = "blue") +
          labs(title = "Training Loss per Batch", x = "Batch", y = "Loss") +
          theme_minimal()
      )
     }
   )

   model %>% compile(
     optimizer = "rmsprop",
     loss = "mse",
     metrics = c("mae")
    )


    history <- model %>% fit(
      x_train,
      y_train,
      epochs =5,
      batch_size = 200,
      validation_data = list(x_test, y_test),
      verbose = 1
      # callbacks = list(class_callback_loss_history())
    )

    plot(history)



Final epoch (plot to see history):
    loss: 0.5
     mae: 0.5663
val_loss: 0.5326
 val_mae: 0.576 

In [32]:

    results <-  model %>% evaluate(x_test, y_test)
    predictions <-  model %>% predict(x_test)

    names(predictions)
    results
    dim(y_test)

    bind_cols(predictions, y_test) %>%  psych::corr.test()

    ggplot(
        data = data.frame(
            cbind(y_test, y_pred = predictions[ , 1])
            ),
        aes(x = y_test, y = y_pred) ) +
        geom_point( alpha = 1/2) +
        geom_smooth(method = "lm") +
        geom_smooth(color = "red")

   resp2[indices[test_indices], ] %>%
       cbind(predictions) %>%
       select(Código, Ma_measure, y_theta_z, predictions) %>%
       group_by(Código) %>%
       summarise_all(.funs = mean) %>%
       select(-Código) %>%
       ggplot(aes(x = Ma_measure, y = predictions) ) +
        geom_point( alpha = 1/2) +
        geom_smooth(method = "lm") +
        geom_smooth(color = "red")


    resp2[indices[test_indices], ] %>%
       cbind(predictions) %>%
       select(Código, Ma_measure, y_theta_z, predictions) %>%
       group_by(Código) %>%
       summarise_all(.funs = mean) %>%
       select(-Código) %>%
       corr.test()


    results <- model %>% evaluate(x_train, y_train)
    predictions <- model %>% predict(x_train)
    cor(predictions, y_train)

    ggplot(
        data = data.frame(
            cbind(y_train, y_pred = predictions[ , 1])
            ),
        aes(x = y_train, y = y_pred) ) +
        geom_point( alpha = 1/2) +
        geom_smooth(color = "orange") +
        geom_smooth(method = "lm")



New names:
• `` -> `...1`
• `` -> `...2`
• `` -> `...3`
• `` -> `...4`
• `` -> `...5`


Call:psych::corr.test(x = .)
Correlation matrix 
       ...1  ...2  ...3  ...4  ...5 O_rec C_rec E_rec A_rec N_rec
...1   1.00 -0.15 -0.03 -0.12  0.08  0.02 -0.03 -0.06  0.00 -0.06
...2  -0.15  1.00  0.51  0.49 -0.02 -0.03  0.04  0.06  0.02  0.03
...3  -0.03  0.51  1.00  0.45  0.11  0.00  0.06  0.09  0.07  0.01
...4  -0.12  0.49  0.45  1.00 -0.09 -0.03  0.02  0.03  0.01  0.02
...5   0.08 -0.02  0.11 -0.09  1.00  0.01 -0.01  0.01  0.02  0.02
O_rec  0.02 -0.03  0.00 -0.03  0.01  1.00  0.07  0.22  0.23 -0.15
C_rec -0.03  0.04  0.06  0.02 -0.01  0.07  1.00  0.18  0.18 -0.26
E_rec -0.06  0.06  0.09  0.03  0.01  0.22  0.18  1.00  0.20 -0.18
A_rec  0.00  0.02  0.07  0.01  0.02  0.23  0.18  0.20  1.00 -0.36
N_rec -0.06  0.03  0.01  0.02  0.02 -0.15 -0.26 -0.18 -0.36  1.00
Sample Size 
[1] 2077
Probability values (Entries above the diagonal are adjusted for multiple tests.) 
      ...1 ...2 ...3 ...4 ...5 O_rec C_rec E_rec A_rec N_rec
...1  0.00 0.00 1.00 0.00 0.00     1  1.00  0.20  1.00  0.11